In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession
    .builder 
    .master("local[*]")
    .appName("test")
    .getOrCreate()
    )

23/05/08 21:12:01 WARN Utils: Your hostname, HONGs-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.0.69 instead (on interface en0)
23/05/08 21:12:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/08 21:12:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark

In [ ]:
df_green = spark.read.parquet('data/input/green/*/*')
df_green = df_green.withColumnRenamed('lpep_pickup_datetime', "pickup_datetime") \
                    .withColumnRenamed('lpep_dropoff_datetime', "dropoff_datetime")

In [ ]:
df_yellow = spark.read.parquet('data/input/yellow/*/*')
df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime', "pickup_datetime") \
                    .withColumnRenamed('tpep_dropoff_datetime', "dropoff_datetime")

In [ ]:
common_columns = []
yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)

In [ ]:
df_green_com = df_green.select(common_columns).withColumn('service_type', F.lit('green'))
df_yellow_com = df_yellow.select(common_columns).withColumn('service_type', F.lit('yellow'))
data = df_green_com.unionAll(df_yellow_com)
data = data.filter(data.pickup_datetime >= '2020-01-01')  # 과거 데이터가 섞여있었음
data = data.filter(data.pickup_datetime < '2023-01-01')   # 이상값 제거

In [ ]:
data.groupBy('service_type').count().show()

In [3]:
df_fhvhv = spark.read.parquet("data/input/fhvhv/*/*")

In [ ]:
df_fhvhv.printSchema()

In [5]:
from pyspark.sql.functions import date_format, asc

In [ ]:
df_fhvhv.withColumn('month', date_format("pickup_datetime", "yyyy-MM")).groupby('month').count().orderBy('month').show()

In [6]:
df_fhvhv = df_fhvhv.withColumn('month', date_format("pickup_datetime", "yyyy-MM"))
df_fhvhv.where(df_fhvhv.month == '2020-01').show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+-------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|  month|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+-------------------

In [9]:
import os
from config.default import *
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GCS_KEY_PATH

In [10]:
from google.cloud import storage

def read_file_blob(path: str) -> str:
    storage_client = storage.Client()
    bucket = storage_client.bucket("2023-de-zoomcamp")
    blob = bucket.blob(path + "/*/*")

    read_output = blob.download_as_string()
    return read_output

In [12]:
read_file_blob("data/taxi_zone_lookup.csv")

NotFound: 404 GET https://storage.googleapis.com/download/storage/v1/b/2023-de-zoomcamp/o/data%2Ftaxi_zone_lookup.csv%2F%2A%2F%2A?alt=media: No such object: 2023-de-zoomcamp/data/taxi_zone_lookup.csv/*/*: ('Request failed with status code', 404, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)

In [ ]:
a